

```
# This is formatted as code
```



In [29]:
import json
import pandas as pd
import json
import requests

In [99]:
stock=input("Enter Ticker Symbol(s): ")
# Remove spaces in case you put a list of tickers
stock = stock.replace(" ","")

Enter Ticker Symbol(s): AAPL, MSFT, KO


In [121]:
apikey="YqbUGqTj8JlLWTdAcVXc7cz0QsaVfmv76LM9URT9"

url_quote="https://yfapi.net/v6/finance/quote"

url_trending="https://yfapi.net/v1/finance/trending/US"

url_target="https://yfapi.net/ws/insights/v1/finance/insights"

# Split because the assignment asks for only one stock
split = stock.split(",")

# Pass the first stock as the querystring
querystring={"symbols":split[0]}
target_querystring={"symbol":split[0]}

headers={
    "x-api-key":apikey
}

In [122]:
# Stock quote
quote_response = requests.request("GET", url_quote, headers=headers, params=querystring)
target_response = requests.request("GET", url_target, headers=headers, params=target_querystring)

print(quote_response.text)
print(target_response.text)

{"quoteResponse":{"result":[{"language":"en-US","region":"US","quoteType":"EQUITY","typeDisp":"Equity","quoteSourceName":"Nasdaq Real Time Price","triggerable":true,"customPriceAlertConfidence":"HIGH","currency":"USD","exchange":"NMS","shortName":"Apple Inc.","longName":"Apple Inc.","messageBoardId":"finmb_24937","exchangeTimezoneName":"America/New_York","exchangeTimezoneShortName":"EDT","gmtOffSetMilliseconds":-14400000,"market":"us_market","esgPopulated":false,"regularMarketChangePercent":0.09010277,"regularMarketPrice":227.725,"postMarketChangePercent":-0.127307,"postMarketTime":1727481596,"postMarketPrice":227.5,"postMarketChange":-0.289993,"regularMarketChange":0.20500183,"regularMarketTime":1727467204,"regularMarketDayHigh":229.52,"regularMarketDayRange":"227.3 - 229.52","regularMarketDayLow":227.3,"regularMarketVolume":34025967,"regularMarketPreviousClose":227.52,"bid":227.59,"ask":240.09,"bidSize":5,"askSize":1,"fullExchangeName":"NasdaqGS","financialCurrency":"USD","regularMar

In [123]:
stock_quote_json = quote_response.json()
stock_target_json = target_response.json()

# check if the API call returned bad values
if (len(stock_quote_json['quoteResponse']['result'])!=0 and len(stock_quote_json['quoteResponse']['result'])!=None):
    print("Stock symbol: "+stock_quote_json['quoteResponse']['result'][0]["symbol"]+"; "
          "Full name: "+str(stock_quote_json['quoteResponse']['result'][0]["longName"])+"; "
          "Current market price: "+str(stock_quote_json['quoteResponse']['result'][0]["regularMarketPrice"])+"; "
          "Target mean price: " +str(stock_target_json['finance']['result']['instrumentInfo']['recommendation']["targetPrice"])+"; "
          # Use above when API call to insights page goes through. I keep getting the "Missing Authentication Token" error.
          "52-week high: "+str(stock_quote_json['quoteResponse']['result'][0]["fiftyTwoWeekHigh"])+"; "
          "52-week low: "+str(stock_quote_json['quoteResponse']['result'][0]["fiftyTwoWeekLow"])
          )
elif (len(stock_quote_json['quoteResponse']['result'])==0 or len(stock_quote_json['quoteResponse']['result'])==None):
    print("Invalid ticker symbol.")

Stock symbol: AAPL; Full name: Apple Inc.; Current market price: 227.725; Target mean price: 250.0; 52-week high: 237.23; 52-week low: 164.08


In [124]:
# Trending
trending_response = requests.request("GET", url_trending, headers=headers)
trending_json = trending_response.json()
print(trending_json)

{'finance': {'result': [{'count': 20, 'quotes': [{'symbol': 'AMSC'}, {'symbol': 'PWR'}, {'symbol': 'SNAP'}, {'symbol': 'SOL-USD'}, {'symbol': 'GC=F'}, {'symbol': 'NQ=F'}, {'symbol': 'VRT'}, {'symbol': 'BBD'}, {'symbol': 'EME'}, {'symbol': 'CL=F'}, {'symbol': '^N225'}, {'symbol': 'ES=F'}, {'symbol': '9988.HK'}, {'symbol': '^IXIC'}, {'symbol': 'ROKU'}, {'symbol': 'JPY=X'}, {'symbol': 'BTC-USD'}, {'symbol': 'BTC=F'}, {'symbol': 'SI=F'}, {'symbol': 'PATH'}], 'jobTimestamp': 1727648593880, 'startInterval': 202409292100}], 'error': None}}


In [125]:
print("Top 5 trending stocks: ")
for index in range(5):
    print(trending_json['finance']['result'][0]['quotes'][index]['symbol'])

Top 5 trending stocks: 
AMSC
PWR
SNAP
SOL-USD
GC=F


In [126]:
# create arrays
symbols = []
longName = []
regularMarketPrice = []
fiftyTwoWeekHigh = []
fiftyTwoWeekLow = []

# populate arrays for dataframe, used for loop originally meant for more than one ticker
for index in range(len(stock_quote_json['quoteResponse']['result'])):
    symbols.append(stock_quote_json['quoteResponse']['result'][index]["symbol"])
    longName.append(stock_quote_json['quoteResponse']['result'][index]["longName"])
    regularMarketPrice.append(stock_quote_json['quoteResponse']['result'][index]["regularMarketPrice"])
    fiftyTwoWeekHigh.append(stock_quote_json['quoteResponse']['result'][index]["fiftyTwoWeekHigh"])
    fiftyTwoWeekLow.append(stock_quote_json['quoteResponse']['result'][index]["fiftyTwoWeekLow"])

# put in dataframe
stocks = pd.DataFrame({
    'Symbol' : symbols,
    'Full name' : longName,
    'Current market price' : regularMarketPrice,
    '52-week high' : fiftyTwoWeekHigh,
    '52-week low' : fiftyTwoWeekLow
})

#  export to csv
stocks.to_csv('stocks.csv')

# repeat for the trending stocks
trending_symbols = []
for index in range(5):
    trending_symbols.append(trending_json['finance']['result'][0]['quotes'][index]['symbol'])

trending_stocks = pd.DataFrame({
    'Symbol' : trending_symbols
})

trending_stocks.to_csv('trending_stocks.csv')

